In [ ]:
!pip install torch
!pip install duckdb
!pip install graphviz
!pip install semopy
!pip install psy
!pip install pygam
!pip install wbdata

In [5]:
import os
from synthetic_exp import *
from preprocess import *

In [3]:
runs = 100
num_nodes = [10, 50, 100, 500]
# mi_thresholds = [0.005, 0.01, 0.02, 0.05, 0.1]
mi_thresholds = [0.02]
gen_synthetic_data(runs, num_nodes, 10000)

In [11]:
runs = 100
num_nodes = [10, 50, 100]
mi_thresholds = [0.02]
all_res_suna, gt_dict= accuracy_exp(
    runs=runs, num_nodes=num_nodes, mi_thresholds=mi_thresholds, factorized_hist=True, gpu=True)

Treatment: V3, Outcome: V5, Error: 1.262368275913304e-07, Estimation: 2.6237188539138545, Ground Truth: 2.624074152137311
Treatment: V3, Outcome: V5, Error: 0.0, Estimation: 3.272913085285953, Ground Truth: 3.272913085285953
Treatment: V4, Outcome: V5, Error: 0.0, Estimation: -0.00040197919605490225, Ground Truth: -0.00040197919605490225
Treatment: V4, Outcome: V9, Error: 0.0, Estimation: 0.1994621089520616, Ground Truth: 0.1994621089520616
Treatment: V5, Outcome: V8, Error: 5.75040532773723e-07, Estimation: 1.3688248622024826, Ground Truth: 1.3695831764729363
Treatment: V5, Outcome: V7, Error: 7.895018954477894e-18, Estimation: 0.013188708491799345, Ground Truth: 0.013188711301606981
Treatment: V3, Outcome: V6, Error: 0.0, Estimation: 0.0066701691331114, Ground Truth: 0.0066701691331114
Treatment: V5, Outcome: V8, Error: 4.770191698530736e-16, Estimation: -0.1672742626117427, Ground Truth: -0.16727424077097416
Treatment: V5, Outcome: V8, Error: 0.0, Estimation: 0.023902431553334404, G

In [12]:
all_res_suna

{10: {0.02: {'se': 0.046554127758587215,
   'size': 145,
   'preprocess': 76.24506783485413,
   'end_to_end': 13.122114419937134,
   'search': 3.575129747390747,
   'update_res': 0.2255871295928955}},
 50: {0.02: {'se': 0.2825997187403517,
   'size': 789,
   'preprocess': 76.0426733493805,
   'end_to_end': 65.01640129089355,
   'search': 16.046019554138184,
   'update_res': 1.6546299457550049}},
 100: {0.02: {'se': 0.6771994629317486,
   'size': 1692,
   'preprocess': 76.01000928878784,
   'end_to_end': 146.29849481582642,
   'search': 41.97637414932251,
   'update_res': 4.436155080795288}}}

In [4]:
# Experiment for Stack Overflow dataset
# We preprocess the dataset
df = pd.read_csv("SO_integrated.csv")

df = df[df['Age'].isin([9, 21, 29])]
features = [col for col in df.columns if col not in {'Country', 'Age', 'Currency'}]

for col in features:
    df = remove_outliers(df, col)

treatment = 'EdLevel'
outcome = 'CompTotal'

df['ID'] = range(1, len(df) + 1)
buyer_df = df[['ID', 'Country', treatment, outcome]]


seller_1_features = ['gini_index']
seller_2_features = ['Human Development Index (HDI) ', 'Expected years of schooling', 
                     'Mean years of schooling', 'HDI rank']
seller_3_features = ['YearsCode', 'ConvertedCompYearly', 'Rate']
seller_df_1 = df[['Country'] + seller_1_features].drop_duplicates()
seller_df_2 = df[['Country'] + seller_2_features].drop_duplicates()
seller_df_3 = df[['ID'] + seller_3_features]

ID_domain = buyer_df['ID'].values
country_domain = list(buyer_df['Country'].unique())

In [5]:
causal_effect(buyer_df, X=treatment, Y=outcome)

-9516.986902482622

In [6]:
dm = DataMarket()
dm.add_seller(seller_df_1, "GINI", [['Country']], 
              {'Country': country_domain}, seller_1_features)
dm.add_seller(seller_df_2, "HDI", [['Country']], 
              {'Country': country_domain}, seller_2_features)
dm.add_seller(seller_df_3, "SO", [['ID']], 
              {'ID': ID_domain}, seller_3_features)

for data in os.listdir('so_datasets'):
    df = pd.read_csv(f'so_datasets/{data}')
    # jk_domain = set(list(df['country_standardized'].unique()))
    # jk_intersection = jk_domain.intersection(country_domain)
    preprocess = Preprocess()
    preprocess.load(df, 'country_standardized')
    preprocess.agg_by_jk()
    preprocess.data = preprocess.data.rename(
        columns={'country_standardized': 'Country'})
    dm.add_seller(
        preprocess.data, data, [['Country']], {'Country': country_domain}, preprocess.X)

In [7]:
import pickle

with open('dm_country.pkl', 'wb') as file:
    pickle.dump(dm, file)

In [8]:
cd = ConDiscovery(dm, mi_threshold=0.05, verbose=False, factorized_hist=True)
target_ce, _, _, _, _, _, ate_so = cd.compute_treatment_effect(
    buyer_df, [['ID'], ['Country']], treatment, outcome)

In [9]:
ate_so

[-9516.986902482628,
 -353.9076636895113,
 3426.5220731642944,
 3477.0016845295877,
 3424.0561864202687,
 3284.946704395135,
 3453.7066241111793,
 4973.77519625425,
 5136.865708592961,
 5198.136150498652,
 5491.861941303282,
 5551.854660882427,
 5571.848023338191,
 5588.433067338872,
 5569.5434924061055,
 5563.601861580063,
 5596.956408975614,
 5598.978650550357,
 5541.203010788577,
 5528.206811618528,
 5534.280093782337,
 5542.976587183421,
 5540.596757388481]

In [10]:
with open('dm_country.pkl', 'rb') as file:
    dm = pickle.load(file)
prune_confounders(buyer_df, cd, dm, target_ce, treatment, outcome, k=2)

(['standardized_population_by_country_2020.csv:Land Area (Km²)',
  'HDI:Human Development Index (HDI) '],
 4615.718419397591)

In [98]:
def create_random_datasets(domain_size, num_rows, m, seed):
    np.random.seed(seed)
    join_key_domain = range(domain_size)
    domain_size = len(join_key_domain)
    
    repetitions = num_rows // domain_size
    
    # Create the join key column
    join_key = np.tile(join_key_domain, repetitions)
    
    # Create the datasets
    df1 = pd.DataFrame({
        'join_key': join_key,
        'T': np.random.rand(num_rows),
        'O': np.random.rand(num_rows)
    })
    
    df2_dict = {'join_key': join_key}
    for i in range(m):
        df2_dict[f'V{i}'] = np.random.rand(num_rows)
    df2 = pd.DataFrame(df2_dict)
    
    return df1, df2

import torch.nn.functional as F

def compute_kl_divergence(p_fhm_fwd, p_hist_fwd, epsilon=1e-6):
    p_fhm_fwd = p_fhm_fwd.float()
    p_hist_fwd = p_hist_fwd.float()
    
    # Add small epsilon to avoid log(0)
    p_fhm_fwd = p_fhm_fwd + epsilon
    p_hist_fwd = p_hist_fwd + epsilon
    
    # Renormalize after adding epsilon
    p_fhm_fwd = p_fhm_fwd / p_fhm_fwd.sum(dim=0, keepdim=True)
    p_hist_fwd = p_hist_fwd / p_hist_fwd.sum(dim=0, keepdim=True)
    
    # Compute KL divergence for each column
    kl_div = F.kl_div(p_hist_fwd.log(), p_fhm_fwd, reduction='none').sum(dim=0)
    
    return kl_div

In [99]:
import copy
device = 'cpu'
num_sample = 10000

df1, df2 = create_random_datasets(num_sample, num_sample, 10, seed)
df1_copy, df2_copy = copy.deepcopy(df1), copy.deepcopy(df2)
dm = DataMarket()
dm.add_seller(df2_copy, "synthetic", [['join_key']], {'join_key': np.arange(num_sample)}, 
              [col for col in df2_copy if col != 'join_key'])
cd = ConDiscovery(dm, hist=True, factorized_hist=False, device=device, test_histogram=True)
_, _, _, _, _, _, _ = cd.compute_treatment_effect(df1_copy, [['join_key']], 'T', 'O')

df1_copy, df2_copy = copy.deepcopy(df1), copy.deepcopy(df2)
dm = DataMarket()
dm.add_seller(df2_copy, "synthetic", [['join_key']], {'join_key': np.arange(num_sample)}, 
              [col for col in df2_copy if col != 'join_key'])
cd = ConDiscovery(dm, hist=False, factorized_hist=True, device=device, test_histogram=True)
_, _, _, _, _, _, _ = cd.compute_treatment_effect(df1_copy, [['join_key']], 'T', 'O')

kl_forward = []
kl_backward = []

with open(f'fhm_fwd_{num_sample}.pkl', 'rb') as file:
    fhm_fwd = pickle.load(file)
with open(f'hist_fwd_{num_sample}.pkl', 'rb') as file:
    hist_fwd = pickle.load(file)

with open(f'fhm_bwd_{num_sample}.pkl', 'rb') as file:
    fhm_bwd = pickle.load(file)
with open(f'hist_bwd_{num_sample}.pkl', 'rb') as file:
    hist_bwd = pickle.load(file)

p_fhm_fwd = fhm_fwd/num_sample
p_hist_fwd = hist_fwd/num_sample
p_fhm_bwd = fhm_bwd/num_sample
p_hist_bwd = hist_bwd/num_sample
divergence_fwd = compute_kl_divergence(p_fhm_fwd, p_hist_fwd, epsilon=1e-10)
divergence_bwd = compute_kl_divergence(p_fhm_bwd, p_hist_bwd, epsilon=1e-10) 

In [100]:
divergence_fwd

tensor([0.1102, 0.0105, 0.0540, 0.0495, 0.5717, 0.0356, 0.5873, 0.3239, 0.3002,
        0.0346])

In [101]:
divergence_bwd

tensor([0.2193, 0.0887, 0.0402, 0.0580, 0.0871, 0.0619, 0.0784, 0.2239, 0.2073,
        0.0633])